In [5]:
import pandas as pd

# 1. Convert the raw 23andMe data to vcf

We will remove all SNPs corresponding to deletions and insertions, to make the file compatible with annotation tools

`../plink_mac_20201019/plink --23file data/SNP_raw_v4_Full_20170514175358.txt --recode vcf --out snps_clean --output-chr MT --snps-only just-acgt`

We interested only clinically relevant SNPs, so we will intersect result with vcf from ClinVar database (https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh37/clinvar.vcf.gz).

Download and unzip:

`wget https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh37/clinvar.vcf.gz`

`gzip -d clinvar.vcf.gz`

Intersect:

`bedtools intersect -a snps_clean.vcf -b clinvar.vcf  > intersected.vcf`

# 2. Annotation

We will use VEP (Variant Effect Predictor) online version (http://grch37.ensembl.org/Homo_sapiens/Tools/VEP).

Input:

<img src="img/vep.png">

Result in `data/vep_result.txt`. Now look at result:

In [95]:
df = pd.read_csv('data/vep_result.txt', sep='\t')
df.head()

,#Uploaded_variation,Location,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,...,AF,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS
0,rs2843159,1:2235672-2235672,T,intron_variant,MODIFIER,SKI,ENSG00000157933,Transcript,ENST00000378536.4,protein_coding,...,0.3498,-,-,-,"16327884,22984993",-,-,-,-,-
1,rs2843159,1:2235672-2235672,T,downstream_gene_variant,MODIFIER,SKI,ENSG00000157933,Transcript,ENST00000478223.2,processed_transcript,...,0.3498,-,-,-,"16327884,22984993",-,-,-,-,-
2,rs2843159,1:2235672-2235672,T,"intron_variant,non_coding_transcript_variant",MODIFIER,SKI,ENSG00000157933,Transcript,ENST00000507179.1,retained_intron,...,0.3498,-,-,-,"16327884,22984993",-,-,-,-,-
3,rs2843159,1:2235672-2235672,T,downstream_gene_variant,MODIFIER,SKI,ENSG00000157933,Transcript,ENST00000508416.1,processed_transcript,...,0.3498,-,-,-,"16327884,22984993",-,-,-,-,-
4,rs2234167,1:2494330-2494330,G,missense_variant,MODERATE,TNFRSF14,ENSG00000157873,Transcript,ENST00000355716.4,protein_coding,...,-,-,-,-,-,-,-,-,-,-


Now we will look at CLIN_SIG and try to find pathogenic and risk_factor SNPs

In [96]:
clin_sigs = set()

for c in df['CLIN_SIG'].unique():
    if type(c) == str:
        c_a = c.split('|')
        clin_sigs.update(c_a)

clin_sigs

{'-',
 'affects',
 'association',
 'association,benign',
 'association_not_found',
 'benign',
 'benign,_other,benign',
 'benign,affects',
 'benign,benign/likely_benign',
 'benign,benign/likely_benign,pathogenic',
 'benign,drug_response',
 'benign,likely_benign',
 'benign,likely_benign,other',
 'benign,likely_pathogenic',
 'benign,not_provided',
 'benign,not_provided,drug_response',
 'benign,not_provided,likely_benign',
 'benign,pathogenic',
 'benign,risk_factor',
 'benign/likely_benign',
 'benign/likely_benign,benign',
 'benign/likely_benign,benign,likely_benign',
 'conflicting_interpretations_of_pathogenicity,benign',
 'drug_response',
 'drug_response,benign',
 'drug_response,benign,_other',
 'drug_response,benign,risk_factor',
 'drug_response,benign/likely_benign,benign,likely_benign',
 'drug_response,likely_benign,benign',
 'drug_response,not_provided',
 'likely_benign',
 'likely_benign,benign',
 'likely_benign,benign,risk_factor',
 'likely_benign,benign/likely_benign,pathogenic,ben

We will look at pathogenic tag

In [102]:
search_for = ['pathogenic', 'risk_factor']
pathogenic_df = df[df['CLIN_SIG'].str.contains('|'.join(search_for))].groupby('#Uploaded_variation').first()
pathogenic_df = pathogenic_df[['Location', 'Allele', 'Codons', 'CLIN_SIG']]
pathogenic_df.CLIN_SIG

#Uploaded_variation
i3000469                                            risk_factor
i5005436                                             pathogenic
i5006568      likely_benign,benign/likely_benign,pathogenic,...
i6007787                                     risk_factor,benign
i6015290      uncertain_significance,likely_benign,conflicti...
i6015729                 pathogenic,benign/likely_benign,benign
i6058143                       drug_response,benign,risk_factor
i6058764                                  protective,pathogenic
i6059141                       likely_benign,benign,risk_factor
i6060296      conflicting_interpretations_of_pathogenicity,b...
rs10151259         not_provided,pathogenic,likely_benign,benign
rs1024611                                pathogenic,risk_factor
rs1042503                              likely_pathogenic,benign
rs1049296                        risk_factor,association,benign
rs11085825                             benign,likely_pathogenic
rs11558492          

In [103]:
pathogenic_df

,Location,Allele,Codons,CLIN_SIG
#Uploaded_variation,,,,
i3000469,2:138759649-138759649,T,aCa/aTa,risk_factor
i5005436,6:32008312-32008312,T,-,pathogenic
i5006568,2:71829924-71829924,G,Atc/Gtc,"likely_benign,benign/likely_benign,pathogenic,..."
i6007787,2:234183368-234183368,G,Act/Gct,"risk_factor,benign"
i6015290,14:23887607-23887607,T,aaC/aaA,"uncertain_significance,likely_benign,conflicti..."
i6015729,1:156848918-156848918,T,Cat/Tat,"pathogenic,benign/likely_benign,benign"
i6058143,1:161479745-161479745,G,cAt/cGt,"drug_response,benign,risk_factor"
i6058764,16:27356203-27356203,G,Atc/Gtc,"protective,pathogenic"
i6059141,8:133909974-133909974,G,Atg/Gtg,"likely_benign,benign,risk_factor"


We will find this SNPs in dbSNP https://www.ncbi.nlm.nih.gov/snp/

# 3. mtDNA haplogroup.

To define mtDNA haplogroup we will use: https://dna.jameslick.com/mthap/

Results:

mthap version 0.19b (2015-05-11); haplogroup data version PhyloTree Build 17 (2016-02-18) +mods 
raw data source SNP_raw_v4_Full_20170514175358.txt (14MB)

Found 3270 markers at 3268 positions covering 19.7% of mtDNA.


Markers found (shown as differences to rCRS):

HVR2: 152C 263G
CR: 750G 1438G 4769G 8860G
HVR1:

IMPORTANT NOTE: The above marker list is almost certainly incomplete due to limitations of genotyping technology and is not comparable to mtDNA sequencing results. It should not be used with services or tools that expect sequencing results, such as mitosearch.


Best mtDNA Haplogroup Matches:

1) H(T152C)

Defining Markers for haplogroup H(T152C):
HVR2: 152C 263G
CR: 750G 1438G 4769G 8860G 15326G
HVR1:

Marker path from rCRS to haplogroup H(T152C):
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 152C ⇨ H(T152C)

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Untested(1): 15326


2) H1(T152C)

Defining Markers for haplogroup H1(T152C):
HVR2: 152C 263G
CR: 750G 1438G 3010A 4769G 8860G 15326G
HVR1:

Marker path from rCRS to haplogroup H1(T152C):
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 3010A ⇨ H1 ⇨ 152C ⇨ H1(T152C)

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
No-Calls(1): 3010A
Untested(1): 15326


3) H

Defining Markers for haplogroup H:
HVR2: 263G
CR: 750G 1438G 4769G 8860G 15326G
HVR1:

Marker path from rCRS to haplogroup H (plus extra markers):
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 152C

Imperfect Match. Your results contained differences with this haplogroup:
Matches(5): 263G 750G 1438G 4769G 8860G
Extras(1): 152C
Untested(1): 15326


3) H69

Defining Markers for haplogroup H69:
HVR2: 152C 263G
CR: 750G 1438G 4646C 4769G 8860G 15326G
HVR1:

Marker path from rCRS to haplogroup H69:
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 152C ⇨ H(T152C) ⇨ 4646C ⇨ H69

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Mismatches(1): 4646T
Untested(1): 15326


3) H3(T152C)

Defining Markers for haplogroup H3(T152C):
HVR2: 152C 263G
CR: 750G 1438G 4769G 6776C 8860G 15326G
HVR1:

Marker path from rCRS to haplogroup H3(T152C):
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 6776C ⇨ H3 ⇨ 152C ⇨ H3(T152C)

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Mismatches(1): 6776T
Untested(1): 15326


3) H16(T152C)

Defining Markers for haplogroup H16(T152C):
HVR2: 152C 263G
CR: 750G 1438G 4769G 8860G 10394T 15326G
HVR1:

Marker path from rCRS to haplogroup H16(T152C):
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 10394T ⇨ H16 ⇨ 152C ⇨ H16(T152C)

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Mismatches(1): 10394C
Untested(1): 15326


3) H9

Defining Markers for haplogroup H9:
HVR2: 152C 263G
CR: 750G 1438G 4769G 8860G 13020C 15326G
HVR1:

Marker path from rCRS to haplogroup H9:
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 152C ⇨ H(T152C) ⇨ 13020C ⇨ H9

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Mismatches(1): 13020T
Untested(1): 15326


3) H46

Defining Markers for haplogroup H46:
HVR2: 152C 263G
CR: 750G 1438G 2772T 4769G 8860G 15326G
HVR1:

Marker path from rCRS to haplogroup H46:
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 152C ⇨ H(T152C) ⇨ 2772T ⇨ H46

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Mismatches(1): 2772C
Untested(1): 15326


3) H52

Defining Markers for haplogroup H52:
HVR2: 152C 263G
CR: 750G 1438G 4769G 8860G 14220G 15326G
HVR1:

Marker path from rCRS to haplogroup H52:
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 152C ⇨ H(T152C) ⇨ 14220G ⇨ H52

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Mismatches(1): 14220A
Untested(1): 15326

# 4.  Y-chromosome haplogroup

To define Y-chromosome haplogroup we will use: https://ytree.morleydna.com/extractFromAutosomal

Result:

<img src="img/y.png">

# 5. Nationality.

We will use https://github.com/stevenliuyi/admix to define nationality

In [104]:
!pip install admix

     |████████████████████████████████| 37.6 MB 233 kB/s eta 0:00:01
  Using cached scipy-1.6.1-cp38-cp38-macosx_10_9_x86_64.whl (30.8 MB)
Using legacy 'setup.py install' for admix, since package 'wheel' is not installed.
    Running setup.py install for admix ... done


In [121]:
!admix -f data/SNP_raw_v4_Full_20170514175358.txt -v 23andme -m K7b K12b globe13 world9 E11


Admixture calculation models: K7b,K12b,globe13,world9,E11

Calcuation is started...

K7b
South Asian: 2.47%
West Asian: 25.71%
Siberian: 2.00%
African: 0.12%
Southern: 21.19%
Atlantic Baltic: 48.50%
East Asian: 0.00%


K12b
Gedrosia: 8.22%
Siberian: 1.57%
Northwest African: 1.83%
Southeast Asian: 0.00%
Atlantic Med: 18.38%
North European: 34.10%
South Asian: 1.73%
East African: 0.00%
Southwest Asian: 8.18%
East Asian: 0.00%
Caucasus: 25.99%
Sub Saharan: 0.00%


globe13
Siberian: 1.43%
Amerindian: 0.25%
West African: 0.00%
Palaeo African: 0.27%
Southwest Asian: 12.85%
East Asian: 0.00%
Mediterranean: 24.47%
Australasian: 0.91%
Artic: 0.54%
West Asian: 20.26%
North European: 37.56%
South Asian: 1.45%
East African: 0.02%


world9
Amerindian: 0.22%
East Asian: 0.00%
African: 0.27%
Atlantic Baltic: 48.02%
Australasian: 0.70%
Siberian: 2.02%
Caucasus Gedrosia: 26.30%
Southern: 20.76%
South Asian: 1.71%


E11
African: 2.02%
European: 68.54%
India: 22.17%
Malay: 0.30%
South Chinese Dai: 0.00%